# VPoser poZ Space for Body Models
The original body pose space of [SMPL](http://smpl.is.tue.mpg.de/) and [SMPL-X](https://smpl-x.is.tue.mpg.de/) 
(jointly refered here as SMPL) are not bounded to natural human pose space. That means you can put a vector value 
as the pose of a SMPL body model and get broken body, that might not even look like a human.
To address this you can replace the original pose space of SMPL with VPoser's latent space that corresponds to natural human pose space.
This means if you sample a 32 dimentional random vector from a Normal distribution and pass it as your pose you should get  a viable human joint configuration. This representation of pose is fully differentiable and can be used in an end-to-end deep learning pipeline.

An example use case is an expressive 2D-to-3D lifting of a human image done in [Expressive Body Capture:
3D Hands, Face, and Body from a Single Image](https://smpl-x.is.tue.mpg.de/)
 
To load a SMPL with this body pose representation first you need to obtain a trained VPoser and a variation of SMPL model, here we use SMPLx, from https://smpl-x.is.tue.mpg.de/downloads .
Put the obtained vposer model and body models in a folder, here we assume respectively

'GITHUB_CLONE_ROOT/human_body_prior/dowloads/vposer_v1_0', and 
'GITHUB_CLONE_ROOT/human_body_prior/dowloads/models'

and afterwards you can use the following code snippet to load BodyModel with vposer: 


In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
%matplotlib inline

expr_dir = '../dowloads/vposer_v1_0' #'TRAINED_MODEL_DIRECTORY'  in this directory the trained model along with the model code exist
bm_path =  '../dowloads/models/smplx/SMPLX_FEMALE.npz'#'PATH_TO_SMPLX_model.npz'  obtain from https://smpl-x.is.tue.mpg.de/downloads

In [2]:
from human_body_prior.body_model.body_model_vposer import BodyModelWithPoser


bm = BodyModelWithPoser(bm_path=bm_path, batch_size=1, poser_type='vposer', smpl_exp_dir=expr_dir).to('cuda')

Found Trained Model: ../dowloads/vposer_v1_0/snapshots/TR00_E096.pt


In the above snippet, *bm* is a BodyModel isntance where the body pose is not represented by body_pose anymore, 
but with body_poZ that is the latent space of respective VPoser model. So in any optimization task that involves a human body
you can use bm.body_poZ as the free variable and as a prior put a L2 norm on this vriable to keep it close to the mean of a Normal. Eventually, you can weight this term to choose between data and prior term.

In [3]:
print('poZ_body', bm.poZ_body.shape)
print('pose_body', bm.pose_body.shape)


poZ_body torch.Size([1, 32])
pose_body torch.Size([1, 63])


You can take a look at notebooks/vposer_sampling.ipynb to learn how to sample and visualize vposer novel poses.